In [8]:
import pandas as pd

### 📂 Chargement des données de ventes immobilières

In [7]:
# Chargement des ventes
ventes = pd.read_excel("Données-immo/Valeurs-foncières.xlsx")
ventes.head(2)

,Code service CH,Reference document,1 Articles CGI,2 Articles CGI,3 Articles CGI,4 Articles CGI,5 Articles CGI,No disposition,Date mutation,Nature mutation,...,Nombre de lots,Code type local,Type local,Identifiant local,Surface reelle bati,Nombre pieces principales,Nature culture,Nature culture speciale,Surface terrain,Nom de l'acquereur
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2020-01-02,Vente,...,2,2,Appartement,NaN,48,3,NaN,NaN,NaN,GUIRAO
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2020-01-02,Vente,...,2,2,Appartement,NaN,40,1,NaN,NaN,NaN,HARNOIS


### 📊 Sélection et compréhension des colonnes clés


In [ ]:
# Colonnes importantes à garder
colonnes_importantes = [
    "Code departement",   # Département (ex : 75 = Paris)
    "Code commune",       # Commune à l'intérieur du département
    "Nom commune",        # Nom de la commune
    "Valeur fonciere",    # Prix de vente du bien
    "Type local",         # Maison, appartement, dépendance...
    "Surface reelle bati",# Surface habitable en m²
    "Nature mutation",    # Vente, échange, adjudication...
    "Date mutation"       # Date de la transaction
]

# On garde seulement les colonnes importantes si elles existent
ventes_cle = ventes[[col for col in colonnes_importantes if col in ventes.columns]]

# Aperçu des premières lignes
print("\nAperçu des colonnes importantes :")
print(ventes_cle.head(5))

# Vérification des liens : un département contient plusieurs communes
nb_communes_par_dept = ventes_cle.groupby("Code departement")["Code commune"].nunique().reset_index()
nb_communes_par_dept.columns = ["Code departement", "Nombre de communes"]

print("\nNombre de communes par département :")
print(nb_communes_par_dept.head(10))

# Vérification des valeurs foncières moyennes par département
prix_moyen_par_dept = ventes_cle.groupby("Code departement")["Valeur fonciere"].mean().reset_index()
prix_moyen_par_dept.columns = ["Code departement", "Prix moyen"]

print("\nPrix moyen par département :")
print(prix_moyen_par_dept.sort_values(by="Prix moyen", ascending=False).head(10))



Aperçu des colonnes importantes :
  Code departement Code commune  Valeur fonciere   Type local  \
0               01          103         165000.0  Appartement   
1               06          004         355680.0  Appartement   
2               06          088         229500.0  Appartement   
3               06          123         125000.0  Appartement   
4               13          005          90000.0  Appartement   

   Surface reelle bati Nature mutation Date mutation  
0                   48           Vente    2020-01-02  
1                   40           Vente    2020-01-02  
2                   82           Vente    2020-01-02  
3                   27           Vente    2020-01-02  
4                   47           Vente    2020-01-02  

Nombre de communes par département :
  Code departement  Nombre de communes
0               01                  54
1               02                  19
2               03                   4
3               04                  33
4          

### 🔑 Création d'une clé unique département-commune


In [8]:


# On s'assure que les codes sont bien formatés en texte
ventes["Code departement"] = ventes["Code departement"].astype(str).str.zfill(2)
ventes["Code commune"] = ventes["Code commune"].astype(str).str.zfill(3)

# Création de la clé unique (ex : 75056 = Paris)
ventes["CodeInseeCommune"] = ventes["Code departement"] + ventes["Code commune"]

# Aperçu
print(ventes[["Code departement", "Code commune", "CodeInseeCommune"]].head(10))


  Code departement Code commune CodeInseeCommune
0               01          103            01103
1               06          004            06004
2               06          088            06088
3               06          123            06123
4               13          005            13005
5               13          028            13028
6               13          208            13208
7               13          212            13212
8               14          338            14338
9               14          366            14366


In [ ]:

# 📂 Chargement des fichiers
ventes = pd.read_excel("Données-immo/Valeurs-foncières.xlsx")
communes = pd.read_excel("Données-immo/donnees_communes.xlsx")

# 🔑 Création de la clé unique dans les deux fichiers
ventes["Code departement"] = ventes["Code departement"].astype(str).str.zfill(2)
ventes["Code commune"] = ventes["Code commune"].astype(str).str.zfill(3)
ventes["CodeInseeCommune"] = ventes["Code departement"] + ventes["Code commune"]

communes["Code departement"] = communes["CODDEP"].astype(str).str.zfill(2)
communes["Code commune"] = communes["CODCOM"].astype(str).str.zfill(3)
communes["CodeInseeCommune"] = communes["Code departement"] + communes["Code commune"]

# 🔗 Jointure sur la clé unique
ventes_communes = ventes.merge(
    communes,
    on="CodeInseeCommune",
    how="left"
)

# 📊 Aperçu du résultat
print(ventes_communes.head())


   Code service CH  Reference document  1 Articles CGI  2 Articles CGI  \
0              NaN                 NaN             NaN             NaN   
1              NaN                 NaN             NaN             NaN   
2              NaN                 NaN             NaN             NaN   
3              NaN                 NaN             NaN             NaN   
4              NaN                 NaN             NaN             NaN   

   3 Articles CGI  4 Articles CGI  5 Articles CGI  No disposition  \
0             NaN             NaN             NaN               1   
1             NaN             NaN             NaN               1   
2             NaN             NaN             NaN               1   
3             NaN             NaN             NaN               1   
4             NaN             NaN             NaN               1   

  Date mutation Nature mutation  ...  CODDEP  CODARR CODCAN  CODCOM  \
0    2020-01-02           Vente  ...      01     3.0     20     103  

### 🧹 Nettoyage et préparation des données immobilières

In [2]:
import pandas as pd

# 📂 Chargement des fichiers Excel
ventes_raw = pd.read_excel("Données-immo/Valeurs-foncières.xlsx")
communes_raw = pd.read_excel("Données-immo/donnees_communes.xlsx")
referentiel_geo_raw = pd.read_excel("Données-immo/fr-esr-referentiel-geographique.xlsx")

# 🔧 Nettoyage du fichier "Valeurs-foncières.xlsx"
# - Suppression des lignes avec valeurs manquantes critiques (ex: Valeur fonciere, Date mutation)
ventes_raw = ventes_raw.dropna(subset=["Valeur fonciere", "Date mutation", "Code departement", "Code commune", "Type local"])

# - Conversion des types : Date mutation en datetime, Valeur fonciere en float, codes en str
ventes_raw["Date mutation"] = pd.to_datetime(ventes_raw["Date mutation"], errors='coerce')
ventes_raw["Valeur fonciere"] = pd.to_numeric(ventes_raw["Valeur fonciere"], errors='coerce')
ventes_raw["Code departement"] = ventes_raw["Code departement"].astype(str).str.zfill(2)
ventes_raw["Code commune"] = ventes_raw["Code commune"].astype(str).str.zfill(3)

# - Gestion des NaN dans les colonnes numériques : Remplacer par 0 pour surfaces et pièces
ventes_raw["Surface reelle bati"] = ventes_raw["Surface reelle bati"].fillna(0)
ventes_raw["Nombre pieces principales"] = ventes_raw["Nombre pieces principales"].fillna(0)
ventes_raw["Surface terrain"] = ventes_raw["Surface terrain"].fillna(0)

# - Nettoyage des chaînes : Majuscules pour les adresses, suppression espaces inutiles
ventes_raw["Voie"] = ventes_raw["Voie"].astype(str).str.strip().str.upper()
ventes_raw["Type local"] = ventes_raw["Type local"].astype(str).str.strip()

# - Suppression des doublons basés sur toutes les colonnes
ventes_raw = ventes_raw.drop_duplicates()

print(f"✅ Nettoyage de Valeurs-foncières.xlsx terminé. {len(ventes_raw)} lignes restantes.")

# 🔧 Nettoyage du fichier "donnees_communes.xlsx"
# - Suppression des lignes avec valeurs manquantes critiques (ex: CODDEP, CODCOM, COM, PTOT)
communes_raw = communes_raw.dropna(subset=["CODDEP", "CODCOM", "COM", "PTOT"])

# - Conversion des types : Codes en str zfill, PTOT en int
communes_raw["CODREG"] = communes_raw["CODREG"].astype(str).str.zfill(2)
communes_raw["CODDEP"] = communes_raw["CODDEP"].astype(str).str.zfill(2)
communes_raw["CODCOM"] = communes_raw["CODCOM"].astype(str).str.zfill(3)
communes_raw["PTOT"] = pd.to_numeric(communes_raw["PTOT"], errors='coerce').fillna(0).astype(int)

# - Nettoyage des chaînes : Nom commune en majuscules, suppression espaces
communes_raw["COM"] = communes_raw["COM"].astype(str).str.strip().str.upper()

# - Suppression des doublons basés sur CODDEP + CODCOM
communes_raw = communes_raw.drop_duplicates(subset=["CODDEP", "CODCOM"])

print(f"✅ Nettoyage de donnees_communes.xlsx terminé. {len(communes_raw)} lignes restantes.")

# 🔧 Nettoyage du fichier "fr-esr-referentiel-geographique.xlsx"
# - Suppression des lignes avec valeurs manquantes critiques (ex: reg_nom, dep_nom, com_nom)
referentiel_geo_raw = referentiel_geo_raw.dropna(subset=["reg_nom", "dep_nom", "com_nom"])

# - Conversion des types : Codes en str zfill
referentiel_geo_raw["reg_code"] = referentiel_geo_raw["reg_code"].astype(str).str.zfill(2)
referentiel_geo_raw["dep_code"] = referentiel_geo_raw["dep_code"].astype(str).str.zfill(2)
referentiel_geo_raw["com_code"] = referentiel_geo_raw["com_code"].astype(str).str.zfill(5)  # com_code est souvent sur 5 caractères

# - Nettoyage des chaînes : Noms en majuscules, suppression espaces
referentiel_geo_raw["reg_nom"] = referentiel_geo_raw["reg_nom"].astype(str).str.strip().str.upper()
referentiel_geo_raw["dep_nom"] = referentiel_geo_raw["dep_nom"].astype(str).str.strip().str.upper()
referentiel_geo_raw["com_nom"] = referentiel_geo_raw["com_nom"].astype(str).str.strip().str.upper()

# - Gestion des NaN dans geolocalisation : Remplacer par chaîne vide
referentiel_geo_raw["geolocalisation"] = referentiel_geo_raw["geolocalisation"].fillna("")

# - Suppression des doublons basés sur com_id ou codes uniques
referentiel_geo_raw = referentiel_geo_raw.drop_duplicates(subset=["com_id"])

print(f"✅ Nettoyage de fr-esr-referentiel-geographique.xlsx terminé. {len(referentiel_geo_raw)} lignes restantes.")

# ⚠️ Note : Le fichier "fr-esr-referentiel-geographique.xlsx" n'est pas utilisé dans le code principal fourni,
# mais il a été nettoyé comme demandé. Vous pouvez l'intégrer si nécessaire (ex: pour enrichir la table Commune).

# 📥 Sauvegarde des fichiers nettoyés (optionnel, pour vérification)
ventes_raw.to_excel("Valeurs-foncières_nettoye.xlsx", index=False)
communes_raw.to_excel("donnees_communes_nettoye.xlsx", index=False)
referentiel_geo_raw.to_excel("fr-esr-referentiel-geographique_nettoye.xlsx", index=False)

print("✅ Fichiers nettoyés sauvegardés.")

✅ Nettoyage de Valeurs-foncières.xlsx terminé. 34151 lignes restantes.
✅ Nettoyage de donnees_communes.xlsx terminé. 34991 lignes restantes.
✅ Nettoyage de fr-esr-referentiel-geographique.xlsx terminé. 38916 lignes restantes.
✅ Fichiers nettoyés sauvegardés.


In [3]:
import pandas as pd
import sqlite3

# 📂 Charger les fichiers (après nettoyage)
ventes_raw = pd.read_excel("Valeurs-foncières_nettoye.xlsx")
communes_raw = pd.read_excel("donnees_communes_nettoye.xlsx")
# referentiel_geo_raw = pd.read_excel("fr-esr-referentiel-geographique_nettoye.xlsx")  # Non utilisé, mais chargé si besoin

# 🔑 Création clé INSEE
ventes_raw["Code departement"] = ventes_raw["Code departement"].astype(str).str.zfill(2)
ventes_raw["Code commune"] = ventes_raw["Code commune"].astype(str).str.zfill(3)
ventes_raw["id_codedep_codecommune"] = ventes_raw["Code departement"] + ventes_raw["Code commune"]

communes_raw["Code departement"] = communes_raw["CODDEP"].astype(str).str.zfill(2)
communes_raw["Code commune"] = communes_raw["CODCOM"].astype(str).str.zfill(3)
communes_raw["id_codedep_codecommune"] = communes_raw["Code departement"] + communes_raw["Code commune"]

# 🗂 Table Bien
bien = ventes_raw[[
    "id_codedep_codecommune", "No voie", "B/T/Q", "Code type de voie", "Voie",
    "Nombre pieces principales", "Surface reelle bati", "Surface terrain", "Type local"
]].copy()

bien = bien.rename(columns={
    "No voie": "No_voie",
    "B/T/Q": "BTQ",
    "Code type de voie": "Type_voie",
    "Voie": "Voie",
    "Nombre pieces principales": "Total_piece",
    "Surface reelle bati": "Surface_carrez",
    "Surface terrain": "Surface_local",
    "Type local": "Type_local"
})

bien["Id_bien"] = range(1, len(bien) + 1)
bien = bien[[
    "Id_bien", "id_codedep_codecommune", "No_voie", "BTQ", "Type_voie", "Voie",
    "Total_piece", "Surface_carrez", "Surface_local", "Type_local"
]]

# 🗂 Table Vente
vente = ventes_raw[["Date mutation", "Valeur fonciere"]].copy()
vente = vente.rename(columns={
    "Date mutation": "Date",
    "Valeur fonciere": "Valeur"
})
vente["Id_vente"] = range(1, len(vente) + 1)
vente["Id_bien"] = bien["Id_bien"]
vente = vente[["Id_vente", "Id_bien", "Date", "Valeur"]]

# 🗂 Table Commune (sans Code postal)
commune = communes_raw[[
    "id_codedep_codecommune", "CODREG", "Code departement", "Code commune", "COM", "PTOT"
]].drop_duplicates()

commune = commune.rename(columns={
    "CODREG": "Id_region",
    "Code departement": "Code_departement",
    "Code commune": "Code_commune",
    "COM": "Nom_commune",
    "PTOT": "Nbre_habitant_2019"
})

# 🗂 Table Région
region = communes_raw[["CODREG"]].drop_duplicates()
region = region.rename(columns={"CODREG": "Id_region"})
region["Nom_Region"] = None
region["Nom_regroup"] = None

# 💾 Connexion SQLite
conn = sqlite3.connect("donnees_immo_relationnel.sqlite")

# 📥 Sauvegarde des tables
bien.to_sql("Bien", conn, if_exists="replace", index=False)
vente.to_sql("Vente", conn, if_exists="replace", index=False)
commune.to_sql("Commune", conn, if_exists="replace", index=False)
region.to_sql("Region", conn, if_exists="replace", index=False)

conn.close()
print("✅ Base SQLite créée avec le schéma relationnel complet.")

✅ Base SQLite créée avec le schéma relationnel complet.


In [9]:
import sqlite3

# Connexion à la base
conn = sqlite3.connect("donnees_immo_relationnel.sqlite")
cursor = conn.cursor()

# Exécution de la requête
cursor.execute("""
    SELECT COUNT(*) AS total_appartements_vendus
    FROM Vente v
    JOIN Bien b ON v.Id_bien = b.Id_bien
    WHERE b.Type_local = 'Appartement'
      AND v.Date BETWEEN '2020-01-01' AND '2020-06-30';
""")

# Récupération et affichage du résultat
result = cursor.fetchone()[0]
print(f"Nombre total d'appartements vendus au 1er semestre 2020 : {result}")

# Fermeture de la connexion
conn.close()


Nombre total d'appartements vendus au 1er semestre 2020 : 30922


In [11]:
import sqlite3

# Connexion à la base
conn = sqlite3.connect("donnees_immo_relationnel.sqlite")
cursor = conn.cursor()

# Exécution de la requête
cursor.execute("""
    SELECT r.Nom_region,
           COUNT(*) AS nombre_ventes_appartements
    FROM Vente v
    JOIN Bien b ON v.Id_bien = b.Id_bien
    JOIN Commune c ON b.id_codedep_codecommune = c.id_codedep_codecommune
    JOIN Region r ON c.Id_region = r.Id_region
    WHERE b.Type_local = 'Appartement'
      AND v.Date BETWEEN '2020-01-01' AND '2020-06-30'
    GROUP BY r.Nom_region
    ORDER BY nombre_ventes_appartements DESC;
""")

# Récupération et affichage des résultats
for row in cursor.fetchall():
    print(f"{row[0]} : {row[1]} ventes")

# Fermeture de la connexion
conn.close()


None : 17180 ventes
Île-de-France : 13742 ventes


In [12]:
import sqlite3

# Connexion à la base
conn = sqlite3.connect("donnees_immo_relationnel.sqlite")
cursor = conn.cursor()

# Requête
query = """
    SELECT b.Total_piece AS nb_pieces,
           COUNT(*) AS nombre_ventes,
           ROUND(
               COUNT(*) * 100.0 / (
                   SELECT COUNT(*)
                   FROM Vente v
                   JOIN Bien b2 ON v.Id_bien = b2.Id_bien
                   WHERE b2.Type_local = 'Appartement'
               ), 2
           ) AS proportion_pourcentage
    FROM Vente v
    JOIN Bien b ON v.Id_bien = b.Id_bien
    WHERE b.Type_local = 'Appartement'
    GROUP BY b.Total_piece
    ORDER BY b.Total_piece;
"""

# Exécution et affichage
cursor.execute(query)

print("Nb pièces | Nombre ventes | Proportion (%)")
print("-" * 45)
for nb_pieces, nb_ventes, proportion in cursor.fetchall():
    print(f"{nb_pieces:<9} | {nb_ventes:<14} | {proportion:<.2f}")

# Fermeture
conn.close()


Nb pièces | Nombre ventes | Proportion (%)
---------------------------------------------
0         | 30             | 0.10
1         | 6736           | 21.48
2         | 9773           | 31.16
3         | 8966           | 28.59
4         | 4458           | 14.21
5         | 1114           | 3.55
6         | 203            | 0.65
7         | 54             | 0.17
8         | 17             | 0.05
9         | 8              | 0.03
10        | 2              | 0.01
11        | 1              | 0.00
